In [1]:
import pandas as pd
import gensim

In [2]:
IN_DATA_DIR = "../../data/raw/"
INPUT_FILE_NAME = 'cleaned.parquet'
OUT_DATA_DIR = "../../data/processed/"

In [3]:
df = pd.read_parquet(IN_DATA_DIR + INPUT_FILE_NAME)
df.head()
len(df)

2386

In [4]:
def count_tags(tag_column):
    tags = tag_column.str.replace(', ', ',').str.lower().str.strip()
    joined_tags = tags.str.cat(sep=',').split(',')
    all_tags_w_dup = pd.Series(joined_tags)

    tag_counts = all_tags_w_dup.value_counts()
    tag_list = list(tag_counts.index)
    return tag_counts, tag_list

In [ ]:
# split_tags = tags.str.split(',')
# tag_counts_per_talk = split_tags.apply(len)

tags, tag_list = count_tags(df['tags'])

In [ ]:
#remove ted tags
def remove_ted_tags(tags,tag_list):
    ted_tags=[]
    for tag in tag_list:
        if 'ted' in tag:
            ted_tags.append(tag)
    print(ted_tags)
    tags = tags.drop(labels = ted_tags)
    tag_list = list(tags.index.values)
    return tags, tag_list, ted_tags

def remove_tags(original_tags,remove_list):
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    
    final_tags = []
    for tag in split_tags:
        if tag not in remove_list:
            
            final_tags.append(tag)
    final_string = ','.join(final_tags)
    return final_string

In [ ]:
tags, tag_list, ted_tags = remove_ted_tags(tags,tag_list)

['tedx', 'ted fellows', 'ted brain trust', 'ted prize', 'united states', 'tedyouth', 'tedmed', 'ted books', 'ted residency', 'augmented reality', 'ted en español', 'ted en espanol', 'ted-ed']


In [ ]:
tags


technology         695
science            522
global issues      483
culture            470
design             400
business           329
entertainment      285
health             226
innovation         212
education          206
art                204
society            202
social change      198
communication      185
politics           183
future             181
creativity         174
biology            174
humanity           164
collaboration      163
environment        155
economics          154
medicine           154
brain              148
activism           147
invention          136
community          136
children           135
history            135
health care        130
                  ... 
sleep                3
microsoft            3
trust                3
ants                 3
ptsd                 3
pandemic             3
apes                 3
moon                 2
cello                2
driverless cars      2
grammar              2
anthropocene         2
arts       

In [ ]:
def squashing (x,squash_list):
    original_tags = x
#     print(x)
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in squash_list:
            
            final_tags.append(tag)
    final_string = ','.join(final_tags)
    return final_string

In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
nlp_list = []
for i in range(len(tag_list)):
    nlp_list.append(nlp(tag_list[i]))
# print(nlp_list)

In [ ]:
simlist = []
tagsimcount = [0]*len(tag_list)
tagsimdict = {el:[] for el in tag_list}
for outer in range(len(nlp_list)):
    for inner in range(len(nlp_list)):
        if inner<outer:
            sim = nlp_list[outer].similarity(nlp_list[inner])
            if sim > 0.65:
                ow = tag_list[outer]
                iw = tag_list[inner]
                ow_count = tags.at[ow]
                iw_count = tags.at[iw]
                simlist.append([ow,iw,sim])
                tagsimcount[outer] += 1
                tagsimcount[inner] += 1
                tagsimdict[ow].append(iw)
                tagsimdict[iw].append(ow)
                
items =list(tagsimdict.items())

for k,v in items:
    if v == []:
        del tagsimdict[k]             

# print(simlist)
# print(tagsimcount)
# print(tagsimdict)
sim_tags = list(tagsimdict.keys())

In [ ]:
# replace tags
# GENERALISE: 'education','teaching' -> 'education'
# REMAIN: 'africa', 'asia' -> 'africa', 'asia'

#anything with ted remove hahaha
print(tagsimcount)
print(tagsimdict)

In [ ]:
def sim_counts(a):
#     print(a)
    a_count=tags.at[a]
    all_tags = [a]
    all_count = [a_count]
    b = tagsimdict[a]
    for word in b:
        b_count=tags.at[word]
        all_tags.append(word)
        all_count.append(b_count)
    max_count = max(all_count)
    max_index = all_count.index(max_count)
    major = all_tags[max_index]
    minor = all_tags
    minor.remove(major)
    final = [major]+minor
    return final

In [ ]:
sim_tags_ordered = []
for tag in tag_list:
    if tag in sim_tags:
        sim_tags_ordered.append(tag)
# print(sim_tags_ordered)
sim_tags_reversed = sim_tags_ordered
sim_tags_reversed.reverse()
print(sim_tags_reversed)
print(len(sim_tags),len(sim_tags_reversed))

In [ ]:
final_replacement = {}
for i in range(len(sim_tags_reversed)):
    tag = sim_tags_reversed[i]
    sims = tagsimdict[tag]
    #check if words that main tag is similar too have similar words too
    for sim in sims:
        if sim in final_replacement.keys():
            simlist = final_replacement[sim]
            sims+=simlist
            del final_replacement[sim]
    #remove duplicates
    sims = list(dict.fromkeys(sims))
    final_replacement[tag]=sims
print(final_replacement)

In [ ]:
print(final_replacement.keys())

In [ ]:
def replacing (x):
    original_tags = x
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in final_replacement.keys():
            #tag is a major
            final_tags.append(tag)
        else:
            #tag is a minor
            r = []
            for k,v in final_replacement.items():
                if tag in v:
                    #tag is a minor w/ a major
                    r.append(k)
            if r == []:
                r.append(tag)
            final_tags += r
    final_tags = list(dict.fromkeys(final_tags))
    final_string = ','.join(final_tags)
    return final_string

In [ ]:
df1 = df.copy()
df1['sim_tags'] = df1['tags']
df1['sim_tags'] = df1['sim_tags'].map(lambda x: replacing(x))
df1['sim_tags'] = df1['sim_tags'].map(lambda x: remove_tags(x,ted_tags))
df1 = df1[df1['sim_tags']!='']
df1

In [ ]:
tags1,tag1_list = count_tags(df1['sim_tags'])
tags1

In [ ]:
tag_cutoff05 = int(0.05*len(df1.index))

squashed_tags05 = pd.DataFrame(tags1)
squashed_tags05 = squashed_tags05[(squashed_tags05[0]>tag_cutoff05)]
squash_list05 = list(squashed_tags05.index.values)
print(len(squash_list05))
squashed_tags05

In [ ]:
df2 = df1.copy()
df2['squash29_tags'] = df2['sim_tags']
df2['squash29_tags'] = df2['squash29_tags'].map(lambda x: squashing(x,squash_list05))
df2 = df2[df2['squash29_tags']!='']
df2

In [ ]:
tags2,tag2_list = count_tags(df2['squash29_tags'])
tags2

In [ ]:
tag_cutoff075 = int(0.075*len(df1.index))

squashed_tags075 = pd.DataFrame(tags1)
squashed_tags075 = squashed_tags075[(squashed_tags075[0]>tag_cutoff075)]
squash_list075 = list(squashed_tags075.index.values)
print(len(squash_list075))
squashed_tags075

In [ ]:
df3 = df1.copy()
df3['squash15_tags'] = df3['sim_tags']
df3['squash15_tags'] = df3['squash15_tags'].map(lambda x: squashing(x,squash_list075))
df3 = df3[df3['squash15_tags']!='']
df3

In [ ]:
tags3,tag3_list = count_tags(df3['squash15_tags'])
tags3

In [ ]:
# df1 = df1.reset_index(drop=True)
# df2 = df2.reset_index(drop=True)

In [ ]:
# df2.to_parquet(OUT_DATA_DIR+'cleaned_squashed_final.parquet')
